In [10]:
from pyspark.sql import functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from py4j.protocol import Py4JJavaError
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import explode, col, udf, concat_ws, from_json, lit, array, expr, size
from pyspark.sql.functions import sum as _sum
from pyspark.sql.types import *
import json
import os
from pyspark.sql.types import BooleanType
import sys
import gc
print(sys.version)
# conf = SparkConf()
# sc = SparkContext(conf=conf)
# sc.setLogLevel("ERROR")

# spark = SparkSession.builder.appName("twitter_applications")\
#     .config("spark.sql.files.ignoreCorruptFiles", "true")\
#     .config('spark.executor.memory', '45G')\
#     .config('spark.driver.memory', '80G')\
#     .config('spark.driver.maxResultSize', '10G')\
#     .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3.7.16 (default, Dec 15 2022, 23:24:54) 
[GCC 7.3.1 20180712 (Red Hat 7.3.1-15)]

In [12]:
'''
    Step 3:
    Implement basic PageRank as described in 
    http://www.diva-portal.org/smash/get/diva2:1104337/FULLTEXT01.pdf Section 3.4.1
'''
input_bucket = 's3://cs5344-twitter-project/input/'
raw_df = spark.read \
     .json(input_bucket+"retweeted_tweet_intermediate_full/*.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
raw_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+--------------------+-------------------+-------------------------+-----------------------+----------------------------+---------------------------+--------------------+-------------------------------+------------------------------+--------------------+------------------------+--------------------+-------------------+
|connected_user_single|          created_at|             id_str|in_reply_to_status_id_str|in_reply_to_user_id_str|quoted_original_tweet_id_str|quoted_original_user_id_str|         quoted_time|retweeted_original_tweet_id_str|retweeted_original_user_id_str|      retweeted_time|                    text|        text_hashtag|        user_id_str|
+---------------------+--------------------+-------------------+-------------------------+-----------------------+----------------------------+---------------------------+--------------------+-------------------------------+------------------------------+--------------------+------------------------+-------------

In [14]:
test_list = ['pokémon legends: arceus',  'legend of arceus',
             'legend arceus', 'legends of arceus', 'pokemon legends: arceus', 'arceus']
# regex for filter
regex_pattern = "(?i)" + "|".join(test_list)
regex_pattern_hashtag = "(?i)" + "|".join([x.strip(' ') for x in test_list])

filter_df = raw_df.filter(raw_df.text.rlike(regex_pattern) | raw_df.text_hashtag.rlike(regex_pattern)).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
rdd = filter_df.rdd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# for each user how many times retweet/replay/quote other tweets
user_interact_rdd = rdd.map(lambda x: (x[13], 1))
user_interact_rdd = user_interact_rdd.reduceByKey(lambda a, b: a + b)

# each pair of users (A, B), how many times A retweet this person B
user_pair_rdd = rdd.map(lambda x: ((x[13], x[0]), 1)).reduceByKey(lambda a, b: a + b)
user_pair_rdd = user_pair_rdd.map(lambda x: (x[0][0], x[0][1], x[1]))

# user_pair_rdd.toDF().write.mode('overwrite').csv(output_bucket+'user_pair_aws') #coalesce(1, shuffle = True).
# user_interact_rdd.toDF().write.mode('overwrite').csv(output_bucket+'user_interact_aws') #coalesce(1, shuffle = True).

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
id_name = spark.read.option("header",True).csv(input_bucket+"id_name_dict_aws/*.csv")
id_name.show()
# if user A retweet B, the from_user = A, to_user = B
columns = ['from_user', 'to_user', 'retweet_ratio']
edge_rdd = user_pair_rdd.filter(lambda x: x[0] != x[1]).toDF(columns)  # remove all self pointed edges
# .join(user_interact_rdd).map(lambda x: (x[0], x[1][0][0], float(x[1][0][1]) / float(x[1][1]))) \

all_users = edge_rdd.select("from_user").union(edge_rdd.select("to_user")).distinct() \
    .withColumnRenamed("from_user", "users")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+
|        user_id_str|user_twitter_handle|
+-------------------+-------------------+
|1277653717467172864|          KonateEra|
| 889184194315866113|     ViniFerreirado|
|          296488613|              tenoq|
|1292411311612030977|    kamikaze1226617|
|          224175846|           Edunwa18|
|1181834654317531136|         MKtography|
|1241239744316690432|    SpeedDe86282699|
|          201088131|         Fiberblock|
|         4149402979|        luislara_97|
|         2418331615|            eptileR|
|1484471406985297925|           WabinabG|
|1278814483167633409|          jhendryx3|
| 934943098374823936|          shige_e_5|
|1475408590445989894|         TAzammalik|
|         2613144614|         AUTOMYSTIC|
|1482121649650548736|        alexhaujshu|
|1473274640805994497|    hoki2022bigwinx|
|         3173560541|      AshfordWomens|
|1233591105410625536|          jmfoddidk|
|1293106254165024869|         youkapalot|
+-------------------+-------------

In [18]:
edge_rdd.sort(col('retweet_ratio').desc()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+-------------+
|          from_user|            to_user|retweet_ratio|
+-------------------+-------------------+-------------+
| 700747955536400384|           67544145|            3|
| 888929385365073921|         1556857645|            3|
|           68162785|          327942353|            3|
|1030287610311311365| 751823351132323841|            3|
|1335690532329295873|1217225455041961984|            3|
|          247522396|1389017980516831232|            3|
|         2922572473|         2454149816|            2|
| 778430856322912256|1368936879601553415|            2|
|         3389938816|           36519939|            2|
|           73049252|1365337673074933765|            2|
|         1140353412|         2692210960|            2|
|          242593153|1217225455041961984|            2|
|         3934721656|           36519939|            2|
| 808818993536479233|         2482561542|            2|
| 748511608633237504|1365337673074933765|       

In [19]:
auth_rdd = edge_rdd.select("*", lit(1).alias("auth"))
auth_rdd.sort(col("auth").desc()).show(5)
hub_rdd = edge_rdd.select("*", lit(1).alias("hub"))

for i in range(10):

    t = hub_rdd.withColumn("score", hub_rdd.retweet_ratio * hub_rdd.hub).groupby("to_user").agg(_sum("score").alias("a"))
    norm = t.withColumn("square", t.a**2).agg(_sum("square"))

    # norm = t.rdd.map(lambda x: (1, x[1])).reduceByKey(lambda a, b: a**2 + b**2).collect()[0][1]
    norm =  norm.collect()[0][0]**0.5
    auth_rdd = auth_rdd.join(t, "to_user").select("from_user", "to_user", "retweet_ratio",
                                                  (col("a") / norm).alias("auth"))
    auth_rdd.sort(col("auth").desc()).show(5)
    
    del t
    del norm
    gc.collect()

    # update hub
    t = auth_rdd.withColumn("score", auth_rdd.retweet_ratio * auth_rdd.auth).groupby("from_user").agg(
        _sum("score").alias("h"))

    norm = t.withColumn("square", t.h**2).agg(_sum("square"))
    norm =  norm.collect()[0][0]**0.5
    hub_rdd = hub_rdd.join(t, "from_user").select("from_user", "to_user", "retweet_ratio",
                                                  (col("h") / norm).alias("hub"))
    del t
    del norm
    gc.collect()

auth_result = auth_rdd.join(id_name, id_name.user_id_str == auth_rdd.to_user).sort(col("auth").desc())\
    .select('to_user', 'auth', 'user_twitter_handle').distinct().limit(200)
# auth_result.show(50)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://localhost:8998/sessions/7/statements/19 with error payload: {"msg":"requirement failed: Session isn't active."}


In [20]:
auth_result.sort(col("auth").desc()).show()

VBox()

An error was encountered:
Session 7 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
23/04/04 13:33:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/04 13:34:00 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-172-31-34-15.ap-southeast-1.compute.internal/172.31.34.15:8032
23/04/04 13:34:01 INFO Configuration: resource-types.xml not found
23/04/04 13:34:01 INFO ResourceUtils: Unable to find 'resource-types.xml'.
23/04/04 13:34:01 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
23/04/04 13:34:01 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
23/04/04 13:34:01 INFO Client: Setting up container launch context for our AM
23/04/04 13:34:01 INFO Client: Setting up the launch environment for our AM container
23/04/04 13:34:01 INFO Cli

In [21]:
# write to CSV
auth_result.sort(col("auth").desc()).limit(200).write.mode('overwrite').csv(input_bucket+"test_pokemon_hits")

VBox()

An error was encountered:
Session 7 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
23/04/04 13:33:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/04 13:34:00 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-172-31-34-15.ap-southeast-1.compute.internal/172.31.34.15:8032
23/04/04 13:34:01 INFO Configuration: resource-types.xml not found
23/04/04 13:34:01 INFO ResourceUtils: Unable to find 'resource-types.xml'.
23/04/04 13:34:01 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
23/04/04 13:34:01 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
23/04/04 13:34:01 INFO Client: Setting up container launch context for our AM
23/04/04 13:34:01 INFO Client: Setting up the launch environment for our AM container
23/04/04 13:34:01 INFO Cli

In [22]:
hub_result = hub_rdd.join(id_name, id_name.user_id_str == hub_rdd.from_user).sort(col("hub").desc())\
    .select('from_user', 'hub', 'user_twitter_handle').distinct().limit(200)
# write to CSV
hub_result.sort(col("hub").desc()).limit(200).write.mode('overwrite').csv(input_bucket+"test_pokemon_hits_hubs")

VBox()

An error was encountered:
Session 7 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
23/04/04 13:33:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/04 13:34:00 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-172-31-34-15.ap-southeast-1.compute.internal/172.31.34.15:8032
23/04/04 13:34:01 INFO Configuration: resource-types.xml not found
23/04/04 13:34:01 INFO ResourceUtils: Unable to find 'resource-types.xml'.
23/04/04 13:34:01 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
23/04/04 13:34:01 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
23/04/04 13:34:01 INFO Client: Setting up container launch context for our AM
23/04/04 13:34:01 INFO Client: Setting up the launch environment for our AM container
23/04/04 13:34:01 INFO Cli